## RETRIEVER AND CHAIN

In [1]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('attention.pdf').load()

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
split_text=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=250).split_documents(loader)

In [3]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(split_text,OllamaEmbeddings(model="llama3.2:1b"))

C:\Users\ajith\AppData\Local\Temp\ipykernel_12492\4092243650.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  db = FAISS.from_documents(split_text,OllamaEmbeddings(model="llama3.2:1b"))


In [4]:
query = "An estimation function can be described as mapping a query"
result= db.similarity_search(query)
result[0].page_content

'encoder.\n• Similarly, self-attention layers in the decoder allow each position in the decoder to attend to\nall positions in the decoder up to and including that position. We need to prevent leftward\ninformation flow in the decoder to preserve the auto-regressive property. We implement this\ninside of scaled dot-product attention by masking out (setting to −∞) all values in the input\nof the softmax which correspond to illegal connections. See Figure 2.\n3.3 Position-wise Feed-Forward Networks\nIn addition to attention sub-layers, each of the layers in our encoder and decoder contains a fully\nconnected feed-forward network, which is applied to each position separately and identically. This\nconsists of two linear transformations with a ReLU activation in between.\nFFN(x) = max(0, xW1 + b1)W2 + b2 (2)\nWhile the linear transformations are the same across different positions, they use different parameters'

In [5]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3.2:1b")

C:\Users\ajith\AppData\Local\Temp\ipykernel_12492\676642843.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2:1b")


In [6]:
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template("""Answer the following question based on context. <context>{context}<context> Question:{input}""")

In [7]:
from langchain.chains.combine_documents import create_stuff_documents_chain
doc_chain=create_stuff_documents_chain(llm,prompt)

In [8]:
ret = db.as_retriever()
ret

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000010E87211BE0>, search_kwargs={})

### query->> retriever->>vector store(FAISS)->>LLM[Prompt]->>response

In [9]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(ret,doc_chain)


In [10]:
response = retrieval_chain.invoke({"input":"How can an attention function can be described "})
response['answer']

'The question seems to refer to the way in which the Transformer architecture uses self-attention mechanisms to facilitate communication between different parts of a network, including the encoder and decoder stacks. Specifically, it asks how an attention function can be described.\n\nIn the context of the Transformer model, an attention function is used to selectively focus on certain parts of the input sequence when generating output sequences for the decoder. This process allows the model to capture complex relationships between different positions in the input and output sequences.\n\nIn essence, an attention function takes in a set of vectors (representing the input) and outputs a set of weights or scores that represent which parts of the input should be attended to for each position in the output sequence. These weights are used to compute the weighted sum of the input vectors at each position, effectively "attaching" them to the desired part of the output sequence.\n\nIn other w